In [1]:
import os
import os.path as osp
import time
import math
import torch
import numpy as np

os.chdir('../')

import mmcv    
from mmengine.config import Config, DictAction
from mmengine.logging import print_log
from mmengine.registry import RUNNERS
from mmengine.runner import Runner
from mmengine import fileio
from mmdet3d.utils import replace_ceph_backend
from mmengine.structures import InstanceData
from mmdet.models.layers import inverse_sigmoid

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-y3n8jn3v because the default path (/home/hello/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
print('torch version:', torch.__version__)

torch version: 2.1.2+cu121


In [3]:
# cfg = Config.fromfile('my_projects/CMT/configs/cmt_nus.py')
cfg = Config.fromfile('my_projects/CMT/configs/cmt_coda.py')
# cfg = Config.fromfile('my_projects/BEVFusion/configs/bevfusion_lidar-cam_voxel0075_second_secfpn_8xb4-cyclic-20e_nus-3d.py')
cfg.work_dir = osp.abspath('./test/work_dirs')
runner = Runner.from_cfg(cfg)
# runner.load_checkpoint('models/no_remove_dn.pth')

11/04 18:51:37 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1914266801
    GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU
    CUDA_HOME: /usr/local/cuda-12.5
    NVCC: Cuda compilation tools, release 12.5, V12.5.40
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.1.2+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compu

In [4]:
runner.model

CmtDetector(
  (data_preprocessor): Det3DDataPreprocessor()
  (pts_voxel_encoder): HardSimpleVFE()
  (pts_middle_encoder): SparseEncoder(
    (conv_input): SparseSequential(
      (0): SubMConv3d(4, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
      (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (encoder_layers): SparseSequential(
      (encoder_layer1): SparseSequential(
        (0): SparseBasicBlock(
          (conv1): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
          (bn1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (conv2): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], ou

In [ ]:
for data_batch in runner.test_dataloader:
    break
data_batch = runner.model.data_preprocessor(data_batch, training=False)
batch_inputs_dict = data_batch['inputs']
batch_data_samples = data_batch['data_samples']
imgs = batch_inputs_dict.get('imgs', None)
points = batch_inputs_dict.get('points', None)
img_metas = [item.metainfo for item in batch_data_samples]
gt_bboxes_3d = [item.get('eval_ann_info')['gt_bboxes_3d'] for item in batch_data_samples]
gt_labels_3d = [item.get('eval_ann_info')['gt_labels_3d'] for item in batch_data_samples]

In [5]:
runner.model.pts_bbox_head.bbox_coder

In [ ]:
# 训练过程
runner.model.train()
for data_batch in runner.train_dataloader:
    data_batch = runner.model.data_preprocessor(data_batch, training=True)
    if isinstance(data_batch, dict):
        losses = runner.model(**data_batch, mode='loss')
    elif isinstance(data_batch, (list, tuple)):
        losses = runner.model(*data_batch, mode='loss')
    else:
        raise TypeError()
    break

In [ ]:
losses

In [ ]:
# 验证过程
runner.model.eval()
with torch.no_grad():
    for data_batch in runner.val_dataloader:
        data_batch = runner.model.data_preprocessor(data_batch, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
        runner.val_evaluator.process(data_samples=outputs, data_batch=data_batch)
        break

In [ ]:
pred = outputs[0].get('pred_instances_3d')
pred['scores_3d'].shape, pred['labels_3d'].shape, pred['bboxes_3d'].shape

In [ ]:
runner.model.train()
with torch.no_grad():
    data_batch = next(iter(runner.train_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=True)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]
    gt_bboxes_3d = [item.get('gt_instances_3d')['bboxes_3d'] for item in batch_data_samples]
    gt_labels_3d = [item.get('gt_instances_3d')['labels_3d'] for item in batch_data_samples]
    
    img_feats, pts_feats = runner.model.extract_feat(points, imgs=imgs, img_metas=img_metas)
    outs = runner.model.pts_bbox_head(pts_feats, img_feats, img_metas)

In [ ]:
outs['dn_mask_dict']

In [ ]:
runner.model.eval()
with torch.no_grad():
    data_batch = next(iter(runner.val_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]
    gt_bboxes_3d = [item.get('eval_ann_info')['gt_bboxes_3d'] for item in batch_data_samples]
    gt_labels_3d = [item.get('eval_ann_info')['gt_labels_3d'] for item in batch_data_samples]
    
    img_feats, pts_feats = runner.model.extract_feat(points, imgs=imgs, img_metas=img_metas)
    outs = runner.model.pts_bbox_head(pts_feats, img_feats, img_metas)